# Network Creation

In [ ]:
!pip install torch ./Gym-Wordle

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.distributions.categorical import Categorical

In [2]:
class ActorNetwork(nn.Module):
    """LSTM RNN for generating words for wordle solver"""
    def __init__(self, input_size, hidden_size, projection=1, num_layers=5, dropout_probability=0):
        super().__init__()
        self.network = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout_probability, proj_size=projection, batch_first=True)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        output, (hidden, _) = self.network(x)
        probab = hidden + 1 / 2  # takes output of softmax [-1, 1] and squish to [0, 1]
        return probab 

class CriticNet(nn.Module):
    """Network representing the critic"""
    def __init__(self, in_shape):
        super().__init__()
        self.v_network = nn.Sequential(
            nn.Linear(in_shape, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
)

    def forward(self, x):
        return self.v_network(x)

# Toy Problem for RNN

In [3]:
net = ActorNetwork(1, 32, 26, 5)
input = torch.tensor([[
    [1],
    [2],
    [3],
    [4],
    [5]
]]).float()
print(input.shape)
net(input)

torch.Size([1, 5, 1])


tensor([[[0.2923, 0.1851, 0.4920, 0.3597, 0.8412, 0.6158, 0.1874, 0.5343,
          0.4733, 0.5026, 0.7141, 0.4920, 0.6838, 0.5111, 0.2538, 0.2360,
          0.3221, 0.4001, 0.5565, 0.4178, 0.5576, 0.4621, 0.3749, 0.3180,
          0.4408, 0.3913]],

        [[0.5561, 0.5760, 0.4234, 0.4417, 0.5697, 0.5104, 0.5349, 0.5507,
          0.5039, 0.6347, 0.4169, 0.4416, 0.5717, 0.5164, 0.5957, 0.4740,
          0.5279, 0.4453, 0.5402, 0.5396, 0.4736, 0.4346, 0.4352, 0.5314,
          0.4301, 0.4873]],

        [[0.5211, 0.5036, 0.4526, 0.5146, 0.5261, 0.5181, 0.5902, 0.4431,
          0.5116, 0.4362, 0.4546, 0.5290, 0.4591, 0.4928, 0.3859, 0.5028,
          0.6048, 0.5058, 0.5122, 0.5380, 0.5071, 0.4984, 0.6031, 0.4086,
          0.5445, 0.5672]],

        [[0.5102, 0.5152, 0.5857, 0.4535, 0.5058, 0.4311, 0.5242, 0.5814,
          0.5547, 0.5103, 0.5472, 0.5071, 0.4993, 0.4273, 0.5168, 0.5129,
          0.4913, 0.5279, 0.4239, 0.5279, 0.5121, 0.4617, 0.5231, 0.4680,
          0.5369, 0.5313]

# Figuring out the environment

In [4]:
import gym
import gym_wordle
wordle = gym.make("Wordle-v0")

In [9]:
obs = wordle.reset()
wordle.step(2)
wordle.render()
print(obs)

AARGH
     
     
     
     
     
[[ 1  1 18  7  8  3  3  2  3  3]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]


# Competing A2C solvers

In [ ]:
def train(replay, q_val):
    vals = torch.vstack(replay.vals)
    
    rewards_tensor = torch.tensor(np.asarray(replay.rewards, dtype=np.float32))
    is_terminal_tensor = torch.tensor(np.asarray(replay.dones, dtype=np.int32))
        
    q_vals_tensor = rewards_tensor + discount_factor * q_val * (1 - is_terminal_tensor)
        
    advantage = q_vals_tensor - vals
    
    critic_loss = (advantage ** 2).mean()
    # critic_loss.requires_grad = True
    adam_critic.zero_grad()
    critic_loss.backward(retain_graph=True)
    adam_critic.step()
    
    
    log_probabs_tensor = torch.vstack(replay.log_probs)
    actor_loss = (-log_probabs_tensor*advantage.detach()).mean()
    # actor_loss.requires_grad = True
    adam_actor.zero_grad()
    actor_loss.backward()
    adam_actor.step()

In [ ]:
class Advantage_ActorCritic():
    def __init__(self, world: gym.Env, policy_net: ActorNetwork, critic_net: CriticNet,
                 encoder, policy_alpha, critic_alpha, gamma, max_reward):
        # environment info
        self.world = world
        self.encoder = encoder
        self.max_reward = max_reward

        # actor and critic
        self.actor = policy_net
        self.critic = critic_net
        self.error_buffer = list()
        self.policy_optimizer = torch.optim.Adam(policy_net.network.parameters(), lr=policy_alpha)
        self.v_optimizer = torch.optim.Adam(critic_net.v_network.parameters(), lr=critic_alpha)

        # training info
        self.gamma = gamma
        self.episodes = 0

    def train(self, iterations):
        converged = False
        rewards = list()
        recents = torch.zeros(10)
        i = 0
        while not converged:
            r = self.episode()
            rewards.append(r)

            if i < 10:
                recents[i] = r
                i += 1

            else:
                recents.roll(-1, 0)
                recents[9] = r
            
            # convergence check
            if len(rewards) > 10 and recents.mean() >= self.max_reward:
                converged = True
            converged = True if iterations == self.episodes else False
        
        plt.plot(rewards)
        plt.show()


    def episode(self, training=True):
        done = False
        state = self.world.reset().copy()
        episode_reward = 0
        self.episodes += 1
        while not done:
            # take on policy action
            encoded = self.encoder(state)
            net_out = self.actor(encoded)

            dist = Categorical(net_out)
            action = dist.sample()
            try:
                state_prime, reward, done, _ = self.world.step(action)
            except AssertionError:
                reward = -.1
                state_prime = state.copy()

            # fill buffer
            self.error_buffer.append((state, state_prime, reward, dist.log_prob(action)))

            # prepare for next iteration
            episode_reward += reward
            state = state_prime.copy()
            if training:
                self.__net_update()

        return episode_reward

    def td_error(self, value, value_prime, reward):
        def loss_fn():
            print(value.shape, value_prime.shape) 
            return reward + self.gamma * value_prime - value
        return loss_fn()

    def policy_error(self, prob, error):
        def loss_fn():
            print(prob)
            print(error)
            return -torch.log(prob) * error
        return loss_fn()

    def __net_update(self):
        # calculate error for policy and critic
        self.critic.v_network.zero_grad()
        self.actor.network.zero_grad()

        state, state_prime, reward, prob = self.error_buffer.pop(0)
        print(self.encoder(state))
        value = self.critic(self.encoder(state))
        value_prime = self.critic(self.encoder(state_prime))

        td_error = self.td_error(value, value_prime, reward)
        policy_error = self.policy_error(prob, td_error) 

        # backprop
        if self.episodes > 100:
            td_error.backward()
        else:
            policy_error.backward()
            self.policy_optimizer.step()
        self.v_optimizer.step()

# Solving the Environment

In [ ]:
critic = CriticNet(30)
actor = ActorNetwork(30, 32, 26)
agent = Advantage_ActorCritic(wordle, actor, critic, 
lambda x: torch.tensor([x['board']]).to(torch.float32).reshape(1, 1,30), 1e-3, 1e-3, 0.9, 1)